In [2]:
import pickle
import numpy as np
import pandas as pd
from lightgbm import LGBMRanker

In [3]:
def handle_time(df):
    end = df.trip_end_date.astype(int) // 10**9
    start = df.trip_start_date.astype(int) // 10**9
    
    df.drop(['trip_end_date', 'trip_start_date'], axis=1, inplace=True)
    df['trip_length_seconds'] = end - start

In [4]:
# Reading in ze data
with open('data/X_test.pickle', 'rb') as cornichon:
    X_test = pickle.load(cornichon)
with open('data/X_train.pickle', 'rb') as cornichon:
    X_train = pickle.load(cornichon)
with open('data/X_val.pickle', 'rb') as cornichon:
    X_val = pickle.load(cornichon)
with open('data/y_test.pickle', 'rb') as cornichon:
    y_test = pickle.load(cornichon)
with open('data/y_train.pickle', 'rb') as cornichon:
    y_train = pickle.load(cornichon)
with open('data/y_val.pickle', 'rb') as cornichon:
    y_val = pickle.load(cornichon)
with open('data/submission_df_preprocessed.pickle', 'rb') as cornichon:
    submission_df = pickle.load(cornichon)

drop_cols = ['date_time', 'date']
X_test.drop(drop_cols, axis=1, inplace=True)
X_train.drop(drop_cols, axis=1, inplace=True)
X_val.drop(drop_cols, axis=1, inplace=True)
submission_df.drop(drop_cols, axis=1, inplace=True)

handle_time(X_test)
handle_time(X_train)
handle_time(X_val)
handle_time(submission_df)

In [5]:
qids_train = X_train.groupby('srch_id')['srch_id'].count().to_numpy()
qids_val = X_val.groupby('srch_id')['srch_id'].count().to_numpy()

In [6]:
clf = LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        n_estimators=2000,
        learning_rate=0.05,
        random_state=42,
        seed=42,
        boosting="dart",
    )

In [7]:
%%time
clf.fit(
    X = X_train,
    y = y_train,
    group = qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    verbose=10,
)

/Users/nedim.azar/DM/Data_Mining_assignment_1/venv/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: max_position
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[10]	valid_0's ndcg@1: 0.183119	valid_0's ndcg@2: 0.254233	valid_0's ndcg@3: 0.300768	valid_0's ndcg@4: 0.334251	valid_0's ndcg@5: 0.359284
[20]	valid_0's ndcg@1: 0.187104	valid_0's ndcg@2: 0.259542	valid_0's ndcg@3: 0.30511	valid_0's ndcg@4: 0.338924	valid_0's ndcg@5: 0.363854
[30]	valid_0's ndcg@1: 0.188297	valid_0's ndcg@2: 0.260915	valid_0's ndcg@3: 0.30702	valid_0's ndcg@4: 0.339914	valid_0's ndcg@5: 0.366214
[40]	valid_0's ndcg@1: 0.18975	valid_0's ndcg@2: 0.261524	valid_0's ndcg@3: 0.309	valid_0's ndcg@4: 0.341946	valid_0's ndcg@5: 0.366532
[50]	valid_0's ndcg@1: 0.190907	valid_0's ndcg@2: 0.262576	valid_0's ndcg@3: 0.309925	valid_0's ndcg@4: 0.342466	valid_0's ndcg@5: 0.368015
[60]	valid_0's ndcg@1: 0.190503	valid_0's ndcg@2: 0.

[580]	valid_0's ndcg@1: 0.197049	valid_0's ndcg@2: 0.272348	valid_0's ndcg@3: 0.31829	valid_0's ndcg@4: 0.351962	valid_0's ndcg@5: 0.376431
[590]	valid_0's ndcg@1: 0.197197	valid_0's ndcg@2: 0.272838	valid_0's ndcg@3: 0.318651	valid_0's ndcg@4: 0.352091	valid_0's ndcg@5: 0.376499
[600]	valid_0's ndcg@1: 0.197197	valid_0's ndcg@2: 0.27274	valid_0's ndcg@3: 0.318428	valid_0's ndcg@4: 0.352252	valid_0's ndcg@5: 0.376593
[610]	valid_0's ndcg@1: 0.196502	valid_0's ndcg@2: 0.272717	valid_0's ndcg@3: 0.317737	valid_0's ndcg@4: 0.351855	valid_0's ndcg@5: 0.376431
[620]	valid_0's ndcg@1: 0.196853	valid_0's ndcg@2: 0.273489	valid_0's ndcg@3: 0.31889	valid_0's ndcg@4: 0.352668	valid_0's ndcg@5: 0.377197
[630]	valid_0's ndcg@1: 0.196954	valid_0's ndcg@2: 0.273193	valid_0's ndcg@3: 0.31877	valid_0's ndcg@4: 0.352392	valid_0's ndcg@5: 0.377158
[640]	valid_0's ndcg@1: 0.197553	valid_0's ndcg@2: 0.273232	valid_0's ndcg@3: 0.318735	valid_0's ndcg@4: 0.352856	valid_0's ndcg@5: 0.377281
[650]	valid_0's n

[1170]	valid_0's ndcg@1: 0.203339	valid_0's ndcg@2: 0.278043	valid_0's ndcg@3: 0.324932	valid_0's ndcg@4: 0.357988	valid_0's ndcg@5: 0.383509
[1180]	valid_0's ndcg@1: 0.202341	valid_0's ndcg@2: 0.278251	valid_0's ndcg@3: 0.324629	valid_0's ndcg@4: 0.357694	valid_0's ndcg@5: 0.383121
[1190]	valid_0's ndcg@1: 0.202544	valid_0's ndcg@2: 0.278355	valid_0's ndcg@3: 0.324639	valid_0's ndcg@4: 0.357976	valid_0's ndcg@5: 0.383311
[1200]	valid_0's ndcg@1: 0.202642	valid_0's ndcg@2: 0.278251	valid_0's ndcg@3: 0.324747	valid_0's ndcg@4: 0.357798	valid_0's ndcg@5: 0.383249
[1210]	valid_0's ndcg@1: 0.202146	valid_0's ndcg@2: 0.278288	valid_0's ndcg@3: 0.324506	valid_0's ndcg@4: 0.357675	valid_0's ndcg@5: 0.383224
[1220]	valid_0's ndcg@1: 0.202496	valid_0's ndcg@2: 0.278358	valid_0's ndcg@3: 0.324444	valid_0's ndcg@4: 0.357764	valid_0's ndcg@5: 0.383287
[1230]	valid_0's ndcg@1: 0.202246	valid_0's ndcg@2: 0.278179	valid_0's ndcg@3: 0.324561	valid_0's ndcg@4: 0.357787	valid_0's ndcg@5: 0.383143
[1240]

[1750]	valid_0's ndcg@1: 0.205689	valid_0's ndcg@2: 0.280604	valid_0's ndcg@3: 0.327184	valid_0's ndcg@4: 0.360703	valid_0's ndcg@5: 0.385437
[1760]	valid_0's ndcg@1: 0.20544	valid_0's ndcg@2: 0.280548	valid_0's ndcg@3: 0.326911	valid_0's ndcg@4: 0.360815	valid_0's ndcg@5: 0.385465
[1770]	valid_0's ndcg@1: 0.206088	valid_0's ndcg@2: 0.280801	valid_0's ndcg@3: 0.327186	valid_0's ndcg@4: 0.360977	valid_0's ndcg@5: 0.385814
[1780]	valid_0's ndcg@1: 0.205789	valid_0's ndcg@2: 0.280725	valid_0's ndcg@3: 0.326907	valid_0's ndcg@4: 0.360793	valid_0's ndcg@5: 0.38553
[1790]	valid_0's ndcg@1: 0.205893	valid_0's ndcg@2: 0.280602	valid_0's ndcg@3: 0.327065	valid_0's ndcg@4: 0.360586	valid_0's ndcg@5: 0.385423
[1800]	valid_0's ndcg@1: 0.205345	valid_0's ndcg@2: 0.280584	valid_0's ndcg@3: 0.326831	valid_0's ndcg@4: 0.360491	valid_0's ndcg@5: 0.385269
[1810]	valid_0's ndcg@1: 0.205291	valid_0's ndcg@2: 0.280541	valid_0's ndcg@3: 0.32668	valid_0's ndcg@4: 0.360072	valid_0's ndcg@5: 0.385546
[1820]	va

LGBMRanker(boosting='dart', learning_rate=0.05, max_position=5, metric='ndcg',
           n_estimators=2000, objective='lambdarank', random_state=42, seed=42)

In [8]:
%%time
clf_goss = LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        n_estimators=2000,
        learning_rate=0.05,
        max_position=5,
        random_state=42,
        seed=42,
        boosting="goss",
    )

clf_goss.fit(
    X = X_train,
    y = y_train,
    group = qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    verbose=10,
)

[10]	valid_0's ndcg@1: 0.183159	valid_0's ndcg@2: 0.254386	valid_0's ndcg@3: 0.299531	valid_0's ndcg@4: 0.33327	valid_0's ndcg@5: 0.358099
[20]	valid_0's ndcg@1: 0.189911	valid_0's ndcg@2: 0.260298	valid_0's ndcg@3: 0.306071	valid_0's ndcg@4: 0.339978	valid_0's ndcg@5: 0.365778
[30]	valid_0's ndcg@1: 0.19215	valid_0's ndcg@2: 0.262676	valid_0's ndcg@3: 0.309135	valid_0's ndcg@4: 0.342555	valid_0's ndcg@5: 0.368678
[40]	valid_0's ndcg@1: 0.192048	valid_0's ndcg@2: 0.264254	valid_0's ndcg@3: 0.311509	valid_0's ndcg@4: 0.344199	valid_0's ndcg@5: 0.369734
[50]	valid_0's ndcg@1: 0.191997	valid_0's ndcg@2: 0.265473	valid_0's ndcg@3: 0.311984	valid_0's ndcg@4: 0.345454	valid_0's ndcg@5: 0.370277
[60]	valid_0's ndcg@1: 0.193	valid_0's ndcg@2: 0.267176	valid_0's ndcg@3: 0.313215	valid_0's ndcg@4: 0.346451	valid_0's ndcg@5: 0.371681
[70]	valid_0's ndcg@1: 0.195147	valid_0's ndcg@2: 0.267593	valid_0's ndcg@3: 0.314775	valid_0's ndcg@4: 0.347541	valid_0's ndcg@5: 0.372683
[80]	valid_0's ndcg@1: 0.

[600]	valid_0's ndcg@1: 0.204294	valid_0's ndcg@2: 0.280444	valid_0's ndcg@3: 0.326172	valid_0's ndcg@4: 0.360622	valid_0's ndcg@5: 0.385462
[610]	valid_0's ndcg@1: 0.204343	valid_0's ndcg@2: 0.280336	valid_0's ndcg@3: 0.32592	valid_0's ndcg@4: 0.36048	valid_0's ndcg@5: 0.385386
[620]	valid_0's ndcg@1: 0.204191	valid_0's ndcg@2: 0.280215	valid_0's ndcg@3: 0.325981	valid_0's ndcg@4: 0.360428	valid_0's ndcg@5: 0.385477
[630]	valid_0's ndcg@1: 0.204195	valid_0's ndcg@2: 0.280233	valid_0's ndcg@3: 0.325674	valid_0's ndcg@4: 0.360186	valid_0's ndcg@5: 0.385358
[640]	valid_0's ndcg@1: 0.203994	valid_0's ndcg@2: 0.280012	valid_0's ndcg@3: 0.325598	valid_0's ndcg@4: 0.360004	valid_0's ndcg@5: 0.385251
[650]	valid_0's ndcg@1: 0.203696	valid_0's ndcg@2: 0.279519	valid_0's ndcg@3: 0.325453	valid_0's ndcg@4: 0.359823	valid_0's ndcg@5: 0.385339
[660]	valid_0's ndcg@1: 0.203397	valid_0's ndcg@2: 0.279673	valid_0's ndcg@3: 0.325452	valid_0's ndcg@4: 0.359771	valid_0's ndcg@5: 0.385229
[670]	valid_0's

[1190]	valid_0's ndcg@1: 0.203395	valid_0's ndcg@2: 0.279336	valid_0's ndcg@3: 0.325789	valid_0's ndcg@4: 0.360879	valid_0's ndcg@5: 0.385167
[1200]	valid_0's ndcg@1: 0.203395	valid_0's ndcg@2: 0.279462	valid_0's ndcg@3: 0.325829	valid_0's ndcg@4: 0.360914	valid_0's ndcg@5: 0.385111
[1210]	valid_0's ndcg@1: 0.203445	valid_0's ndcg@2: 0.279349	valid_0's ndcg@3: 0.325658	valid_0's ndcg@4: 0.36097	valid_0's ndcg@5: 0.385125
[1220]	valid_0's ndcg@1: 0.203346	valid_0's ndcg@2: 0.279259	valid_0's ndcg@3: 0.32576	valid_0's ndcg@4: 0.360937	valid_0's ndcg@5: 0.384956
[1230]	valid_0's ndcg@1: 0.203495	valid_0's ndcg@2: 0.279268	valid_0's ndcg@3: 0.325932	valid_0's ndcg@4: 0.360979	valid_0's ndcg@5: 0.384975
[1240]	valid_0's ndcg@1: 0.20374	valid_0's ndcg@2: 0.279407	valid_0's ndcg@3: 0.326011	valid_0's ndcg@4: 0.361141	valid_0's ndcg@5: 0.385048
[1250]	valid_0's ndcg@1: 0.203589	valid_0's ndcg@2: 0.279213	valid_0's ndcg@3: 0.325923	valid_0's ndcg@4: 0.360972	valid_0's ndcg@5: 0.385057
[1260]	va

[1770]	valid_0's ndcg@1: 0.203304	valid_0's ndcg@2: 0.279555	valid_0's ndcg@3: 0.326698	valid_0's ndcg@4: 0.360238	valid_0's ndcg@5: 0.384803
[1780]	valid_0's ndcg@1: 0.203154	valid_0's ndcg@2: 0.27947	valid_0's ndcg@3: 0.326733	valid_0's ndcg@4: 0.360157	valid_0's ndcg@5: 0.384775
[1790]	valid_0's ndcg@1: 0.203053	valid_0's ndcg@2: 0.279525	valid_0's ndcg@3: 0.326691	valid_0's ndcg@4: 0.360019	valid_0's ndcg@5: 0.384657
[1800]	valid_0's ndcg@1: 0.202853	valid_0's ndcg@2: 0.279326	valid_0's ndcg@3: 0.326604	valid_0's ndcg@4: 0.359973	valid_0's ndcg@5: 0.384468
[1810]	valid_0's ndcg@1: 0.203204	valid_0's ndcg@2: 0.279381	valid_0's ndcg@3: 0.326639	valid_0's ndcg@4: 0.36022	valid_0's ndcg@5: 0.38459
[1820]	valid_0's ndcg@1: 0.203104	valid_0's ndcg@2: 0.279576	valid_0's ndcg@3: 0.326426	valid_0's ndcg@4: 0.360069	valid_0's ndcg@5: 0.384551
[1830]	valid_0's ndcg@1: 0.2034	valid_0's ndcg@2: 0.279673	valid_0's ndcg@3: 0.326685	valid_0's ndcg@4: 0.360087	valid_0's ndcg@5: 0.384629
[1840]	vali

LGBMRanker(boosting='goss', learning_rate=0.05, max_position=5, metric='ndcg',
           n_estimators=2000, objective='lambdarank', random_state=42, seed=42)

In [9]:
%%time
clf_x = LGBMRanker(
        objective="rank_xendcg",
        metric="ndcg",
        n_estimators=2000,
        learning_rate=0.08,
        max_position=5,
        random_state=42,
        seed=42,
        boosting="dart",
    )

clf_x.fit(
    X = X_train,
    y = y_train,
    group = qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    verbose=10,
)

[10]	valid_0's ndcg@1: 0.17289	valid_0's ndcg@2: 0.239818	valid_0's ndcg@3: 0.28451	valid_0's ndcg@4: 0.317318	valid_0's ndcg@5: 0.341317
[20]	valid_0's ndcg@1: 0.178581	valid_0's ndcg@2: 0.246028	valid_0's ndcg@3: 0.29063	valid_0's ndcg@4: 0.323821	valid_0's ndcg@5: 0.347214
[30]	valid_0's ndcg@1: 0.180875	valid_0's ndcg@2: 0.248718	valid_0's ndcg@3: 0.293143	valid_0's ndcg@4: 0.325014	valid_0's ndcg@5: 0.349183
[40]	valid_0's ndcg@1: 0.184562	valid_0's ndcg@2: 0.252872	valid_0's ndcg@3: 0.298384	valid_0's ndcg@4: 0.331149	valid_0's ndcg@5: 0.354603
[50]	valid_0's ndcg@1: 0.185412	valid_0's ndcg@2: 0.254582	valid_0's ndcg@3: 0.299907	valid_0's ndcg@4: 0.333262	valid_0's ndcg@5: 0.356772
[60]	valid_0's ndcg@1: 0.186412	valid_0's ndcg@2: 0.255953	valid_0's ndcg@3: 0.301387	valid_0's ndcg@4: 0.333201	valid_0's ndcg@5: 0.358439
[70]	valid_0's ndcg@1: 0.187962	valid_0's ndcg@2: 0.25812	valid_0's ndcg@3: 0.303786	valid_0's ndcg@4: 0.335819	valid_0's ndcg@5: 0.359635
[80]	valid_0's ndcg@1: 0

[600]	valid_0's ndcg@1: 0.197849	valid_0's ndcg@2: 0.272838	valid_0's ndcg@3: 0.317578	valid_0's ndcg@4: 0.352162	valid_0's ndcg@5: 0.376521
[610]	valid_0's ndcg@1: 0.197448	valid_0's ndcg@2: 0.27348	valid_0's ndcg@3: 0.317883	valid_0's ndcg@4: 0.351902	valid_0's ndcg@5: 0.376129
[620]	valid_0's ndcg@1: 0.197094	valid_0's ndcg@2: 0.273287	valid_0's ndcg@3: 0.318094	valid_0's ndcg@4: 0.351662	valid_0's ndcg@5: 0.376318
[630]	valid_0's ndcg@1: 0.197146	valid_0's ndcg@2: 0.273455	valid_0's ndcg@3: 0.317978	valid_0's ndcg@4: 0.351621	valid_0's ndcg@5: 0.376672
[640]	valid_0's ndcg@1: 0.196098	valid_0's ndcg@2: 0.272979	valid_0's ndcg@3: 0.317848	valid_0's ndcg@4: 0.351399	valid_0's ndcg@5: 0.376496
[650]	valid_0's ndcg@1: 0.197047	valid_0's ndcg@2: 0.27332	valid_0's ndcg@3: 0.31842	valid_0's ndcg@4: 0.351729	valid_0's ndcg@5: 0.376908
[660]	valid_0's ndcg@1: 0.197204	valid_0's ndcg@2: 0.273294	valid_0's ndcg@3: 0.318861	valid_0's ndcg@4: 0.352174	valid_0's ndcg@5: 0.377574
[670]	valid_0's 

[1190]	valid_0's ndcg@1: 0.200947	valid_0's ndcg@2: 0.277444	valid_0's ndcg@3: 0.324356	valid_0's ndcg@4: 0.357118	valid_0's ndcg@5: 0.381007
[1200]	valid_0's ndcg@1: 0.200798	valid_0's ndcg@2: 0.277238	valid_0's ndcg@3: 0.324229	valid_0's ndcg@4: 0.356978	valid_0's ndcg@5: 0.380863
[1210]	valid_0's ndcg@1: 0.200548	valid_0's ndcg@2: 0.277416	valid_0's ndcg@3: 0.324057	valid_0's ndcg@4: 0.356836	valid_0's ndcg@5: 0.380958
[1220]	valid_0's ndcg@1: 0.201145	valid_0's ndcg@2: 0.277687	valid_0's ndcg@3: 0.324089	valid_0's ndcg@4: 0.356964	valid_0's ndcg@5: 0.381163
[1230]	valid_0's ndcg@1: 0.201197	valid_0's ndcg@2: 0.277306	valid_0's ndcg@3: 0.323766	valid_0's ndcg@4: 0.357128	valid_0's ndcg@5: 0.381253
[1240]	valid_0's ndcg@1: 0.201396	valid_0's ndcg@2: 0.277799	valid_0's ndcg@3: 0.324082	valid_0's ndcg@4: 0.357309	valid_0's ndcg@5: 0.381716
[1250]	valid_0's ndcg@1: 0.201797	valid_0's ndcg@2: 0.27785	valid_0's ndcg@3: 0.324105	valid_0's ndcg@4: 0.357574	valid_0's ndcg@5: 0.381667
[1260]	

[1770]	valid_0's ndcg@1: 0.202394	valid_0's ndcg@2: 0.278472	valid_0's ndcg@3: 0.324655	valid_0's ndcg@4: 0.357876	valid_0's ndcg@5: 0.382624
[1780]	valid_0's ndcg@1: 0.202341	valid_0's ndcg@2: 0.278607	valid_0's ndcg@3: 0.324838	valid_0's ndcg@4: 0.357727	valid_0's ndcg@5: 0.382654
[1790]	valid_0's ndcg@1: 0.202441	valid_0's ndcg@2: 0.278836	valid_0's ndcg@3: 0.324708	valid_0's ndcg@4: 0.357755	valid_0's ndcg@5: 0.382765
[1800]	valid_0's ndcg@1: 0.20269	valid_0's ndcg@2: 0.27893	valid_0's ndcg@3: 0.324949	valid_0's ndcg@4: 0.358124	valid_0's ndcg@5: 0.382906
[1810]	valid_0's ndcg@1: 0.20294	valid_0's ndcg@2: 0.279412	valid_0's ndcg@3: 0.324993	valid_0's ndcg@4: 0.358274	valid_0's ndcg@5: 0.383009
[1820]	valid_0's ndcg@1: 0.203236	valid_0's ndcg@2: 0.279488	valid_0's ndcg@3: 0.325193	valid_0's ndcg@4: 0.3585	valid_0's ndcg@5: 0.383332
[1830]	valid_0's ndcg@1: 0.202789	valid_0's ndcg@2: 0.279367	valid_0's ndcg@3: 0.325016	valid_0's ndcg@4: 0.358377	valid_0's ndcg@5: 0.383299
[1840]	vali

LGBMRanker(boosting='dart', learning_rate=0.08, max_position=5, metric='ndcg',
           n_estimators=2000, objective='rank_xendcg', random_state=42,
           seed=42)

In [10]:
predictions_clf = clf.predict(submission_df)

In [11]:
predictions_goss = clf_goss.predict(submission_df)

In [12]:
predictions_x = clf_x.predict(submission_df)

In [83]:
predictions

array([-0.59366342, -1.09234092,  1.60536836, ..., -0.60151484,
       -0.16473693, -1.19221794])

In [87]:
submission_df = submission_df[['srch_id', 'prop_id']]

In [89]:
submission_df['ord'] = predictions

/var/folders/sg/n9mvg73s2cldzl8k88kctzgh0000gn/T/ipykernel_27543/2690567593.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['ord'] = predictions


In [92]:
submission_df.sort_values(by=['srch_id', 'ord'], ascending=[True, False], inplace=True)

/var/folders/sg/n9mvg73s2cldzl8k88kctzgh0000gn/T/ipykernel_27543/3928364798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df.sort_values(by=['srch_id', 'ord'], ascending=[True, False], inplace=True)


In [95]:
submission_df.drop(['ord'], axis=1).to_csv('submissions/submission_14.csv', index=False)

In [98]:
with open('pickled_models/LGBM-1.pickle', 'wb') as out_file:
    pickle.dump(clf, out_file)

In [102]:
clf.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('ndcg@1', 0.20107451770636509),
                          ('ndcg@2', 0.27791228048419137),
                          ('ndcg@3', 0.3252187803549771),
                          ('ndcg@4', 0.35792926940784126),
                          ('ndcg@5', 0.38246878023539904)])})

In [99]:
submission_df

,srch_id,prop_id,ord
23,1,99484,0.971761
9,1,54937,0.889943
12,1,61934,0.802006
5,1,28181,0.613808
4,1,24194,0.475922
...,...,...,...
4959177,332787,29018,0.763745
4959182,332787,99509,0.704661
4959181,332787,94437,0.446099
4959178,332787,32019,-0.466457
